In [20]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from nuscenes.utils.geometry_utils import points_in_box

In [2]:
nusc = NuScenes(version='v1.0-trainval', dataroot='/home/mike/workspaces/ECE579/data', verbose=True)

Loading NuScenes tables for version v1.0-trainval ...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 24.8 seconds.
Reverse indexing ...
Done reverse indexing in 6.6 seconds.


In [4]:
#Grab first scence
my_scene = nusc.scene[0]
#grab first sample from scene
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)
my_sample

{'token': 'e93e98b63d3b40209056d129dc53ceee',
 'timestamp': 1531883530449377,
 'prev': '',
 'next': '14d5adfe50bb4445bc3aa5fe607691a8',
 'scene_token': '73030fb67d3c46cfb5e590168088ae39',
 'data': {'RADAR_FRONT': 'bddd80ae33ec4e32b27fdb3c1160a30e',
  'RADAR_FRONT_LEFT': '1a08aec0958e42ebb37d26612a2cfc57',
  'RADAR_FRONT_RIGHT': '282fa8d7a3f34b68b56fb1e22e697668',
  'RADAR_BACK_LEFT': '05fc4678025246f3adf8e9b8a0a0b13b',
  'RADAR_BACK_RIGHT': '31b8099fb1c44c6381c3c71b335750bb',
  'LIDAR_TOP': '3388933b59444c5db71fade0bbfef470',
  'CAM_FRONT': '020d7b4f858147558106c504f7f31bef',
  'CAM_FRONT_RIGHT': '16d39ff22a8545b0a4ee3236a0fe1c20',
  'CAM_BACK_RIGHT': 'ec7096278e484c9ebe6894a2ad5682e9',
  'CAM_BACK': 'aab35aeccbda42de82b2ff5c278a0d48',
  'CAM_BACK_LEFT': '86e6806d626b4711a6d0f5015b090116',
  'CAM_FRONT_LEFT': '24332e9c554a406f880430f17771b608'},
 'anns': ['173a50411564442ab195e132472fde71',
  '5123ed5e450948ac8dc381772f2ae29a',
  'acce0b7220754600b700257a1de1573d',
  '8d7cb5e96cae48c39

In [5]:
#Grab first annotation from sample
my_firstannotationToken = my_sample['anns'][3]
my_firstannotationToken
nusc.get('sample_annotation',my_firstannotationToken)

{'token': '8d7cb5e96cae48c39ef4f9f75182013a',
 'sample_token': 'e93e98b63d3b40209056d129dc53ceee',
 'instance_token': 'f7c451dfcd2a4587b3af322d081dddad',
 'visibility_token': '4',
 'attribute_tokens': ['cb5118da1ab342aa947717dc53544259'],
 'translation': [1018.705, 605.045, 0.731],
 'size': [1.638, 4.25, 1.44],
 'rotation': [0.9930201651831624, 0.0, 0.0, 0.11794469695414371],
 'prev': '',
 'next': '3b3734f6209a4317bb8734bcc1e6305f',
 'num_lidar_pts': 150,
 'num_radar_pts': 2,
 'category_name': 'vehicle.car'}

In [78]:
#Grab sensor data for sample
FCRadarData = nusc.get('sample_data', my_sample['data']['RADAR_FRONT'])
FLRadarData = nusc.get('sample_data', my_sample['data']['RADAR_FRONT_LEFT'])
FRRadarData = nusc.get('sample_data', my_sample['data']['RADAR_FRONT_RIGHT'])
RLRadarData = nusc.get('sample_data', my_sample['data']['RADAR_BACK_LEFT'])
RRRadarData = nusc.get('sample_data', my_sample['data']['RADAR_BACK_RIGHT'])

#SHORT CUT 
#channel = nusc.sample_data[0]['channel']
#FrontRadarData

#REDO THIS FOR EACH RADAR

In [35]:
#nusc.my_check_radar_intersects_with_box(my_sample['data']['RADAR_FRONT'], nsweeps=1)

In [92]:
#GRAB Path to point clouds (pc_path), and all annotation boxes with this sensor data associated with them.
#Note that boxes are transformed into sensor frame, so we need a new list of boxes for each sensor
FC_pc_path, boxesFC, array = nusc.get_sample_data(FCRadarData['token'])
FL_pc_path, boxesFL, array = nusc.get_sample_data(FLRadarData['token'])
FR_pc_path, boxesFR, array = nusc.get_sample_data(FRRadarData['token'])
RL_pc_path, boxesRL, array = nusc.get_sample_data(RLRadarData['token'])
RR_pc_path, boxesRR, array = nusc.get_sample_data(RRRadarData['token'])

In [93]:
#PRINT Box 0 so i can see where it is
mybox_FC = boxesFC[3]
mybox_FL = boxesFL[3]
mybox_FR = boxesFR[3]
mybox_RL = boxesRL[3]
mybox_RR = boxesRR[3]

myannoFC=nusc.get('sample_annotation',mybox_FC.token)
myannoFL=nusc.get('sample_annotation',mybox_FC.token)
myannoFR=nusc.get('sample_annotation',mybox_FC.token)
myannoRL=nusc.get('sample_annotation',mybox_FC.token)
myannoRR=nusc.get('sample_annotation',mybox_FC.token)

print(myannoFC ==myannoFL ==myannoFR ==myannoRL== myannoRR)

True


In [89]:
FC_pc = RadarPointCloud.from_file(FC_pc_path)
FL_pc = RadarPointCloud.from_file(FL_pc_path)
FR_pc = RadarPointCloud.from_file(FR_pc_path)
RL_pc = RadarPointCloud.from_file(RL_pc_path)
RR_pc = RadarPointCloud.from_file(RR_pc_path)

In [72]:
this = pc.points[:3, 0].transpose() # grab all points x,y,z values
this = this.reshape(3,1)
this

array([[ 8.39999962],
       [-5.30000019],
       [ 0.        ]])

In [76]:
this = pc.points[:3, :] # grab all points x,y,z values
numpoints=pc.nbr_points()
#this = this.reshape(3,numpoints)
this

array([[  8.39999962,   6.4000001 ,   9.39999962,  14.60000038,
         15.        ,  15.60000038,  17.20000076,  16.39999962,
         16.20000076,  17.39999962,  21.20000076,  17.        ,
         24.        ,  24.        ,  21.20000076,  22.20000076,
         22.79999924,  30.39999962,  24.        ,  35.        ,
         36.79999924,  36.79999924,  37.        ,  39.59999847,
         34.59999847,  32.40000153,  31.39999962,  40.40000153,
         31.20000076,  33.        ,  36.40000153,  46.20000076,
         67.        ,  72.19999695,  77.40000153,  15.        ],
       [ -5.30000019, -10.5       ,  -9.89999962,   2.29999995,
          7.5       ,  -6.9000001 ,  -5.0999999 ,   7.5       ,
         10.69999981,  11.5       ,   1.29999995, -13.69999981,
          1.29999995,  12.69999981,  17.10000038,  17.89999962,
         20.89999962,   9.10000038,  22.29999924,   0.69999999,
          2.29999995,  -2.70000005,  -4.9000001 ,   3.70000005,
         22.89999962,  26.5       ,  28

In [90]:
FC_points = FC_pc.points[:3, :]
FL_points = FL_pc.points[:3, :]
FR_points = FR_pc.points[:3, :]
RL_points = RL_pc.points[:3, :]
RR_points = RR_pc.points[:3, :]

array([[  8.39999962,   6.4000001 ,   9.39999962,  14.60000038,
         15.        ,  15.60000038,  17.20000076,  16.39999962,
         16.20000076,  17.39999962,  21.20000076,  17.        ,
         24.        ,  24.        ,  21.20000076,  22.20000076,
         22.79999924,  30.39999962,  24.        ,  35.        ,
         36.79999924,  36.79999924,  37.        ,  39.59999847,
         34.59999847,  32.40000153,  31.39999962,  40.40000153,
         31.20000076,  33.        ,  36.40000153,  46.20000076,
         67.        ,  72.19999695,  77.40000153,  15.        ],
       [ -5.30000019, -10.5       ,  -9.89999962,   2.29999995,
          7.5       ,  -6.9000001 ,  -5.0999999 ,   7.5       ,
         10.69999981,  11.5       ,   1.29999995, -13.69999981,
          1.29999995,  12.69999981,  17.10000038,  17.89999962,
         20.89999962,   9.10000038,  22.29999924,   0.69999999,
          2.29999995,  -2.70000005,  -4.9000001 ,   3.70000005,
         22.89999962,  26.5       ,  28

In [77]:
points_in_box(mybox, FC_points, 1)
points_in_box(mybox, FL_points, 1)
points_in_box(mybox, FR_points, 1)
points_in_box(mybox, RL_points, 1)
points_in_box(mybox, RR_points, 1)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [56]:
whatiwant = np.array([[0.0, 0.0, 0.0], [0.5, 0.5, 0.0]]).transpose()
whatiwant

array([[0. , 0.5],
       [0. , 0.5],
       [0. , 0. ]])

In [ ]:
points_in_box(mybox, )